In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.metrics import accuracy_score
import numpy as np
import sys
import re
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, InputLayer


In [2]:
vdata = pd.read_csv("VariantTrainingData.csv")
vdata.head(10)

vdata.loc[vdata["Classified"] == 'VOI', "Classified"] = 0
vdata.loc[vdata["Classified"] == 'VOC', "Classified"] = 1
vdata.loc[vdata["Classified"] == 'VUM', "Classified"] = 2

spikeP = vdata["AA Substitutions"]
lineage = vdata["Lineage"]
clade = vdata["Clade"]
variant = vdata["Classified"]

vdata.head(10)


,Virus name,Accession ID,Collection date,Location,Host,Passage,Specimen,Additional host information,Sequencing technology,Assembly method,Comment,Comment type,Lineage,Clade,AA Substitutions,Variant,Classified
0,hCoV-19/England/CAMC-149B04F/2021,EPI_ISL_1483874,3/30/2021,Europe / United Kingdom / England,Human,Original,NaN,NaN,Illumina NovaSeq,NaN,Gap of 30 nucleotides when compared to the ref...,warn,C.37,GR,"(Spike_L249del,Spike_S247del,Spike_G75V,Spike_...",VOI Lambda GR/452Q.V1 (C.37+C.37.1) first dete...,0
1,hCoV-19/USA/NY-CDC-QDX23179684/2021,EPI_ISL_1491965,3/20/2021,North America / USA / New York,Human,Original,Nasal swab,NaN,Illumina MiSeq,BWA v.7.12,Gap of 30 nucleotides when compared to the ref...,warn,C.37,GR,"(Spike_L249del,Spike_S247del,Spike_G75V,Spike_...",VOI Lambda GR/452Q.V1 (C.37+C.37.1) first dete...,0
2,hCoV-19/USA/CA-CDC-FG-010085/2021,EPI_ISL_1481945,3/13/2021,North America / USA / California,Human,Original,Nasal swab,NaN,Illumina NovaSeq,BWA; iVar 1.3,Gap of 30 nucleotides when compared to the ref...,warn,C.37,GR,"(Spike_L249del,Spike_S247del,Spike_G75V,Spike_...",VOI Lambda GR/452Q.V1 (C.37+C.37.1) first dete...,0
3,hCoV-19/USA/TN-CDC-ASC210017485/2021,EPI_ISL_1491924,3/24/2021,North America / USA / Tennessee,Human,Original,Nasal - Anterior Nares,NaN,Illumina NovaSeq 6000,Dragen COVID Lineage v3.5.1,Gap of 30 nucleotides when compared to the ref...,warn,C.37,GR,"(Spike_L249del,Spike_S247del,Spike_G75V,Spike_...",VOI Lambda GR/452Q.V1 (C.37+C.37.1) first dete...,0
4,hCoV-19/USA/TN-CDC-ASC210016026/2021,EPI_ISL_1513430,3/23/2021,North America / USA / Tennessee,Human,Original,Nasal swab,NaN,Illumina NovaSeq 6000,Dragen COVID Lineage v3.5.1,Gap of 30 nucleotides when compared to the ref...,warn,C.37,GR,"(Spike_L249del,Spike_S247del,Spike_G75V,Spike_...",VOI Lambda GR/452Q.V1 (C.37+C.37.1) first dete...,0
5,hCoV-19/USA/TN-CDC-ASC210016009/2021,EPI_ISL_1513424,3/23/2021,North America / USA / Tennessee,Human,Original,Nasopharyngeal swab,NaN,Illumina NovaSeq 6000,Dragen COVID Lineage v3.5.1,Gap of 30 nucleotides when compared to the ref...,warn,C.37,GR,"(Spike_L249del,Spike_S247del,Spike_G75V,Spike_...",VOI Lambda GR/452Q.V1 (C.37+C.37.1) first dete...,0
6,hCoV-19/USA/AR-CDC-LC0035111/2021,EPI_ISL_1515553,3/23/2021,North America / USA / Arkansas,Human,Original,Nasal swab,NaN,PacBio Sequel II,CLC Genomics,Long stretches of NNNs (6.74% of overall seque...,alert,C.37,GR,"(N_T366I,N_G214C,NSP5_G15S,NSP6_G107del,NSP6_S...",VOI Lambda GR/452Q.V1 (C.37+C.37.1) first dete...,0
7,hCoV-19/USA/AZ-CDC-LC0035336/2021,EPI_ISL_1515705,3/26/2021,North America / USA / Arizona,Human,Original,Nasal swab,NaN,PacBio Sequel II,CLC Genomics,Gap of 30 nucleotides when compared to the ref...,warn,C.37,GR,"(Spike_L249del,Spike_S247del,Spike_G75V,Spike_...",VOI Lambda GR/452Q.V1 (C.37+C.37.1) first dete...,0
8,hCoV-19/Canada/QC-1nIUH-5524359940/2021,EPI_ISL_2975479,4/5/2021,North America / Canada / Quebec,Human,Original,NaN,NaN,Illumina_NexteraFlex,iVar 1.3,Gap of 9 nucleotides when compared to the refe...,info,C.37,GR,"(Spike_G75V,NSP6_S106del,N_R203K,Spike_F490S,S...",VOI Lambda GR/452Q.V1 (C.37+C.37.1) first dete...,0
9,hCoV-19/USA/NY-MSHSPSP-PV35341/2021,EPI_ISL_1709289,3/10/2021,North America / USA / New York / Queens County,Human,Original,NaN,NaN,Illumina MiSeq,"Minimap2, Pilon",Gap of 30 nucleotides when compared to the ref...,warn,C.37,GR,"(Spike_L249del,Spike_S247del,Spike_G75V,Spike_...",VOI Lambda GR/452Q.V1 (C.37+C.37.1) first dete...,0


In [3]:
spikeP

0       (Spike_L249del,Spike_S247del,Spike_G75V,Spike_...
1       (Spike_L249del,Spike_S247del,Spike_G75V,Spike_...
2       (Spike_L249del,Spike_S247del,Spike_G75V,Spike_...
3       (Spike_L249del,Spike_S247del,Spike_G75V,Spike_...
4       (Spike_L249del,Spike_S247del,Spike_G75V,Spike_...
                              ...                        
3995    (Spike_T95I,Spike_E484K,NSP7_S26F,NS3_L106F,NS...
3996    (NSP13_P419S,NSP6_Q160R,Spike_T95I,N_A152G,Spi...
3997    (NSP13_P419S,NSP12_M629I,Spike_T95I,Spike_D950...
3998    (Spike_T95I,NSP1_M85del,Spike_E484K,NSP14_L177...
3999    (Spike_T95I,NSP1_M85del,Spike_E484K,NSP14_L177...
Name: AA Substitutions, Length: 4000, dtype: object

In [4]:
cv=CountVectorizer()

In [5]:
x_train, x_test, y_train, y_test = train_test_split(spikeP, variant, test_size=0.2)

x_traincv = cv.fit_transform(x_train)
x_traincv.toarray()
print(type(x_traincv))
print(cv.get_feature_names_out())
print(y_train.astype('int'))

<class 'scipy.sparse._csr.csr_matrix'>
['e_a36v' 'e_d72g' 'e_d72y' ... 'spike_y449n' 'spike_y505h' 'spike_y660h']
2532    2
23      0
3492    1
3609    0
3607    0
       ..
3659    0
1148    1
3305    1
781     1
3893    0
Name: Classified, Length: 3200, dtype: int32


In [6]:
multiNom = MultinomialNB()
multiNom.fit(x_traincv, y_train.astype('int'))

MultinomialNB()

In [7]:
x_testcv = cv.transform(x_test)
print(multiNom.score(x_testcv, y_test.astype('int')))

x_predict = multiNom.predict(x_testcv)
print(accuracy_score(x_predict, y_test.astype('int')))
r2_score(x_predict,y_test.astype('int'))

0.99875
0.99875


0.9966354747134897

In [8]:
GausNB = GaussianNB()
GausNB.fit(x_traincv.toarray(),y_train.astype('int'))

print(GausNB.score(x_testcv.toarray(),y_test.astype('int')))


0.99875


In [48]:
seq_data =  pd.read_csv("Covseqdata.csv")
gene_seq = seq_data["Sequence"]

print(type(seq_data))
print(gene_seq)
print(type(gene_seq[0]))

<class 'pandas.core.frame.DataFrame'>
0       AGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTC...
1       AGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTC...
2       AGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTC...
3       TTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTCGGCTGCA...
4       AGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTC...
                              ...                        
4683    ACTTTCGATCTCTTGTAGATCTGTTCTCTAAACGAACTTTAAAATC...
4684    TAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATC...
4685    ACTTTCGATCTCTTGTAGATCTGTTCTCTAAACGAACTTTAAAATC...
4686    ACTTTCGATCTCTTGTAGATCTGTTCTCTAAACGAACTTTAAAATC...
4687    ACTTTCGATCTCTTGTAGATCTGTTCTCTAAACGAACTTTAAAATC...
Name: Sequence, Length: 4688, dtype: object
<class 'str'>


In [65]:
#pd.options.mode.chained_assignment = None
def str_toarray(seqstr):
    seqstr = seqstr.lower()
    if('z' not in seqstr):
        seqstr = seqstr + 'n'
    seqstr = re.sub('[^acgt]', 'z', seqstr)
    seqstr = np.array(list(seqstr))
    return seqstr
def one_hot_encode(seqstr):
    lbl_encoder = LabelEncoder()
    lbl_encoder.fit(np.array(['a','c','g','t','z']))
    num_encoded = lbl_encoder.transform(seqstr)
    num_encoded = num_encoded.reshape(len(num_encoded),1)
    onehot_encoder = OneHotEncoder(sparse_output = False, dtype=int)
    one_encoded = onehot_encoder.fit_transform(num_encoded)
    if (np.count_nonzero(seqstr=='z') == 1):
        #print("did not find z")
        one_encoded[len(one_encoded) - 1] = [0, 0, 0, 0, 0]
    return one_encoded
one_hot_encode(str_toarray(gene_seq[0]))
onehot_gene_seq = []
#seq_data.loc[seq_data["Sequence"], "One_hot_seq"] = one_hot_encode(str_toarray())
for count, gene in enumerate(gene_seq):
    onehot_gene_seq.insert(count, one_hot_encode(str_toarray(gene)))
    #seq_data.loc[seq_data["Sequence"], "One_hot_seq"] = one_hot_encode(str_toarray(seq_data["Sequence"]))
seq_data['onehot_encode'] = onehot_gene_seq


In [66]:
seq_data.loc[seq_data["Variant"] == 'VOI', "Classified"] = int(0)
seq_data.loc[seq_data["Variant"] == 'VOC', "Classified"] = int(1)
seq_data.loc[seq_data["Variant"] == 'VUM', "Classified"] = int(2)
 
x_seq_data = seq_data["onehot_encode"]
y_seq_data = seq_data["Classified"]
seq_data.head()


,ID,Sequence,Variant,onehot_encode,Classified
0,hCoV-19/Botswana/R43B69_BHP_2421009581/2021|EP...,AGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTC...,VOC,"[[1, 0, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 0...",1.0
1,hCoV-19/Botswana/R43B68_BHP_121142361/2021|EPI...,AGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTC...,VOC,"[[1, 0, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 0...",1.0
2,hCoV-19/Botswana/R43B15_BHP_000842375/2021|EPI...,AGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTC...,VOC,"[[1, 0, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 0...",1.0
3,hCoV-19/Botswana/R43B33_BHP_AAC25682/2021|EPI_...,TTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTCGGCTGCA...,VOC,"[[0, 0, 0, 1, 0], [0, 0, 0, 1, 0], [0, 1, 0, 0...",1.0
4,hCoV-19/Botswana/R43B70_BHP_4021000195/2021|EP...,AGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTC...,VOC,"[[1, 0, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 0...",1.0


In [76]:
padd_x_seq = pad_sequences(x_seq_data, padding = "post")
print(padd_x_seq.shape)

(4688, 30300, 5)


In [70]:
x_train_seq, x_test_seq, y_train_seq, y_test_seq = train_test_split(x_seq_data, y_seq_data, test_size=0.3)


array([[[1, 0, 0, 0, 0],
        [0, 0, 1, 0, 0],
        [1, 0, 0, 0, 0],
        ...,
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]],

       [[1, 0, 0, 0, 0],
        [0, 0, 1, 0, 0],
        [1, 0, 0, 0, 0],
        ...,
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]],

       [[1, 0, 0, 0, 0],
        [1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0],
        ...,
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]],

       ...,

       [[1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 1, 0, 0, 0],
        ...,
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]],

       [[0, 0, 0, 1, 0],
        [0, 0, 0, 1, 0],
        [0, 0, 1, 0, 0],
        ...,
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]],

       [[0, 1, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [1, 0, 0, 0, 0],
        ...,
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]]

In [ ]:
# implement CNN 
CNNmodel = tf.keras.models.Sequential()
#add convolutional layer
CNNmodel.add(Conv2D(32, kernel_size = (3,3), activation = 'relu', input_shape = (x_train_seq[0].shape,1)))
